In [ ]:
# %matplotlib inline
from __future__ import division, print_function

from perf_plotter import get_filtered_data, flatten, get_diffs, __compare, rgetattr
import data_parser
import argparse
import plot_styles as ps
from plot_styles import wheeler, marker_wheel, color_wheel
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import general_plotting as gp
import numpy as np
import os
import pickle
from optionloop import OptionLoop
from collections import defaultdict
import six

In [ ]:
run = data_parser.run
rundata = data_parser.rundata
mechdata = data_parser.mechdata
data = None
try:
    with open(os.path.join('.', 'data.pickle'), 'rb') as file:
        data = pickle.load(file)
except:
    pass
finally:
    if data is None:
        data = data_parser.parse_data()
        with open(os.path.join('.', 'data.pickle'), 'wb') as file:
            pickle.dump(data, file)
            
mechs = data.keys()

In [ ]:
def separate(data, plot_type='runtime', norm=True, plot_condition_scaling=False, plot_cores=False,
             plot_simd=False, return_diffs=False, **pwkwds):
    diffs, locs, check = get_diffs(flatten(data))
    def __eliminate(diff, fail=True):
        ind = next((i for i in range(len(locs)) if check[locs[i]] == diff), None)
        if not fail and ind is None:
            return
        diffs.pop(ind)
        locs.pop(ind)
    if plot_condition_scaling or plot_cores or plot_simd:
        diffs.append(set(data.keys()))
        locs.append(-1)
        check.append('mechdata.name')
    if plot_cores:
        __eliminate('cores')
        
    oploop = OptionLoop({check[locs[i]]: list(diffs[i]) for i in range(len(locs))})
    rv = []
    for state in oploop:
        subdata = get_filtered_data(data, warn=False, strict=True, **state)
        subdata = flatten(subdata)
        if not subdata:
            continue
        x, y, z = gp.process_data(subdata, plot_type, plot_condition_scaling=plot_condition_scaling,
                                  plot_cores=plot_cores, **pwkwds)
        # get the part of the subdata that is identical across all for later ident.
        sd, _, sc = get_diffs(subdata)
        # and store
        same = {c: rgetattr(subdata[0], c) for c in sc if sc not in sd}
        if plot_condition_scaling or plot_cores or plot_simd:
            same['mechdata.name'] = state['mechdata.name']
        rv.append((same, (np.array(x), np.array(y), np.array(z)), subdata))
    if return_diffs:
        return rv, [check[loc] for loc in locs]
    return rv

In [ ]:
def combine(datalist):
    combined = defaultdict(lambda: [])
    for data in datalist:
        for mech in data:
            combined[mech].extend(data[mech])

    return combined

In [ ]:
class normalizer(object):
    def __init__(self, key):
        self.ymax = defaultdict(lambda: None)
        if isinstance(key, list):
            key = tuple(key)
        self.key = key
    
    def __getitem__(self, state):
        if self.key is None:
            return self.ymax['default']
        elif isinstance(self.key, tuple):
            return self.ymax[tuple(state[k] for k in self.key)]
        return self.ymax[state[self.key]]

    def __setitem__(self, state, value):
        if self.key is None:
            self.ymax['default'] = value
        elif isinstance(self.key, tuple):
            self.ymax[tuple(state[k] for k in self.key)] = value
        else:
            self.ymax[state[self.key]] = value
            
class core_norm(normalizer):
    def __init__(self, _):
        super(core_norm, self).__init__('cores')
        
    def select(self, arr):
        return arr[0]
        
    def __setitem__(self, state, value):
        if self.key is None:
            self.ymax['default'] = self.select(value)
        else:
            self.ymax[state[self.key]] = self.select(value)

In [ ]:
def normalize_per_key(data, key):
    # normalize pre-separated data per a given key in the data
    # first pass -- get y max per x
    ymax = normalizer(key)
    op = np.maximum
    for state, (x, y, z), _ in data:
        if ymax[state] is None:
            ymax[state] = y[:]
        def __appended(current, desired):
            diff = desired.shape[0] - current.shape[0]
            return np.hstack((current, np.zeros(diff)))
        if y.shape[0] < ymax[state].shape[0]:
            # missing dat
            y = __appended(y, ymax[state])
        elif y.shape[0] > ymax[state].shape[0]:
            ymax[state] = __appended(ymax[state], y)
        ymax[state] = op(y, ymax[state])
        
    # second pass -- normalize y & z
    for i in range(len(data)):
        state = data[i][0]
        x, y, z = data[i][1]
        z = (z / y) * (ymax[state][:y.shape[0]] / y)
        y = ymax[state][:y.shape[0]] / y
        data[i] = (state, (x, y, z), data[i][2])
   
    return data

In [ ]:
def normalize_per_mech(data):
    # normalize pre-separated data per mechanism
    return normalize_per_key(data, None)

In [ ]:
def plot(data, namefunc, markerfunc, norm=None, stats=False, **pwargs):
    plotfun = pwargs.pop('plotfun', gp.plot)
    sep = separate(data, return_diffs=stats, **pwargs)
    diffs = None
    if stats:
        sep, diffs = sep
        diffs = name_order(diffs, False)
    if norm is not None:
        sep = norm(sep)
    plot_ind = 0
    for state, subdata, runs in sep:
        if stats:
            def stringify(r):
                if diffs:
                    return ', '.join(['{}={}'.format(d, rgetattr(r, d)) for d in diffs])
                else:
                    return state
            print(stringify(runs[0]), [x for  x in zip(subdata[0], subdata[1])])
        plotfun(*subdata, label=namefunc(state), marker=markerfunc(state), plot_ind=plot_ind)
        plot_ind +=1

In [ ]:
def pretty_name(state, key):
    return ps.pretty_names(key).format(state[key])

In [ ]:
# parameter orderings -- to ensure the legend always contains the same
# parameter -> symbol / color mapping
vectype_order = ['w', 'd', 'par', 'openmp']
descriptor_order = ['haswell', 'srv2', 'v1']
gpu_desc_order = ['gpu', 'srv2-gpu', 'v1-gpu']
vecwidth_order = ['4', '8', '16']
gpu_vecwidth_order = ['64', '128']
conp_order = ['conp', 'conv']
data_order = ['C', 'F']
mech_order = ['H2', 'CH4', 'C2H4', 'IC5H11OH']
sparse_order = ['sparse', 'full']
jac_order = ['jac', 'fdjac']
platform_order = ['hpc', 'srv2', 'v1']

# common labels
reac_label = r'Number of Reactions'
cores_label = r'Number of CPU Cores'
states_label = r'Number of Conditions'
speedup_label = r'Speedup'
efficiency_label = r'$\varepsilon$'
simd_efficiency_label = r'$\varepsilon_{\text{SIMD}}$'
runtime_label = r'Run time (\si{\milli\second} / state)'

# and some common legend locations
ll_mid_right = dict(loc='upper right', bbox_to_anchor=(0.92, 0.62))
legend_top = dict(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
                  mode="expand", borderaxespad=0, ncol=2)
ll_mid_right_long = dict(loc='upper right', bbox_to_anchor=(0.92, 0.72))
ll_mid_righter_long = dict(loc='upper right', bbox_to_anchor=(0.95, 0.72))

from os import getcwd
from os.path import dirname, join
figpath = join(dirname(getcwd()), 'figures')

def name_order(param_list, istup=True):
    fields = run._fields + tuple('mechdata.' + x for x in mechdata._fields)
    return sorted(param_list, key=lambda x: fields.index(x[0] if istup else x))

In [ ]:
def finalize_plot(ylog=None, xlog=None, limits={}, labels={}, is_norm=False, figname='',
                  legend_loc={}, legendmaker=None, plot_cores=False, tight=True,
                  plot_simd=False, crossover=False):
    plt.xlabel(labels.get('x', reac_label if not plot_cores else cores_label))
    if plot_cores:
        ylabel_default = efficiency_label if not crossover else runtime_label
    elif plot_simd:
        ylabel_default = simd_efficiency_label
    else:
        ylabel_default = speedup_label if is_norm else runtime_label
    plt.ylabel(labels.get('y', ylabel_default))
    if 'title' in labels:
        plt.title(labels['title'])
    if 'x' in limits:
        plt.gca().set_xlim(limits['x'])
    if 'y' in limits:
        plt.gca().set_ylim(limits['y'])
    if ylog:
        ylog = ylog if ylog != True else 10
        plt.gca().set_yscale('log', basey=ylog)
    if xlog:
        xlog = xlog if xlog != True else 10
        plt.gca().set_xscale('log', basex=xlog)
    legend = ps.legend_style.copy()
    legend.update(legend_loc)
    handles = legendmaker.__getlegend__()
    if handles:
        plt.legend(handles=handles, **legend)
    else:
        plt.legend(**legend)
    # and finalize
    ax = plt.gca()
    # set tick size
    plt.tick_params(axis='both', which='major', labelsize=ps.tick_font_size)
    plt.tick_params(axis='both', which='minor', labelsize=ps.minor_tick_font_size)
    # and label size
    for item in (ax.title, ax.xaxis.label, ax.yaxis.label):
        item.set_fontsize(ps.title_font_size)
    # and set tight
    if tight:
        plt.tight_layout()

    if figname:
        if not figname.endswith('.pdf'):
            figname += '.pdf'
        plt.savefig(join(figpath, figname), bbox_inches="tight")
        
    plt.show()

In [ ]:
def plot_data_standard(data, namefunc, marker_func, labels={}, ylog=False, xlog=False,
                       limits={}, norm=None, filter_num_conditions=None,
                       figname='', legend_loc={},
                       **pwargs):
    if filter_num_conditions is not None:
        data = filter_num_conditions(data)
    marker_func.reset()
    crossover = pwargs.pop('crossover', False)
    plot(data, namefunc, marker_func, norm=norm, **pwargs)
    finalize_plot(ylog=ylog, xlog=xlog, limits=limits, labels=labels, is_norm=norm is not None,
                  figname=figname, legend_loc=legend_loc, legendmaker=marker_func,
                  plot_cores=pwargs.get('plot_cores', False),
                  plot_simd=pwargs.get('plot_simd', False),
                  crossover=crossover)
    
def plot_data_gpu_scaling(*args, **kwargs):
    if not 'labels' in kwargs:
        kwargs['labels'] = {}
    kwargs['labels']['x'] = states_label
    plot_data_standard(*args, **kwargs)

In [ ]:
def plot_data_gpu_standard(*args, **kwargs):
    def max_num_conds_filter(data):
        warn = False
        for mech in data:
            max_num_cond = max([max(r.num_conditions for r in d.rundata) for d in data[mech]])
            for i in range(len(data[mech])):
                local_max_num_cond = max(r.num_conditions for r in data[mech][i].rundata)
                if local_max_num_cond != max_num_cond and not warn:
                    warn = True
                    print('Warning: {} does not match maximum # of conditions {} for data {}'.format(
                          local_max_num_cond, max_num_cond, data[mech][i]))
                copy = data[mech][i].copy()
                data[mech][i] = copy._replace(
                    rundata=[r for r in copy.rundata if r.num_conditions == local_max_num_cond])
        return data
    plot_data_standard(*args, filter_num_conditions=max_num_conds_filter, **kwargs)

In [ ]:
def scaling(d, const_func=lambda state, x: 1. / x, ymin_select = lambda y: y[0]):
    # normalize pre-separated data per a given key in the data
    # first pass -- get y max per x
    ymin = None
    state, (x, y, z), _ = d
    # minimum is assumed to be first data point (either 1 core, or unvectorized simd)
    ymin = ymin_select(y)

    # second pass -- normalize y & z
    c = const_func(state, x)
    z = (z / y) * (ymin * c / y)
    y = ymin * c / y
    # return updated
    return (state, (x, y, z), d[2])

def plot_data_scaling(*args, **kwargs):
    kwargs['plot_cores'] = True
    scale = kwargs.pop('scale', True)
    if scale:
        kwargs['norm'] = lambda data: [scaling(x) for x in data]
    kwargs['xlog'] = 2
    if not 'limits' in kwargs:
        kwargs['limits'] = {}
    if not 'x' in kwargs['limits']:
        kwargs['limits']['x'] = (None, None)
    kwargs['limits']['x'] = (1.8, kwargs['limits']['x'][1])
    plot_data_standard(*args, **kwargs)
    
def plot_data_scaling_crossover(*args, **kwargs):
    base = kwargs.pop('base', 'Shallow')
    kwargs['scale'] = False
    def plot_intercept(*subdata, label=None, marker=None, plot_ind=None):
        if base not in label:
            gp.plot(*subdata, label=label, marker=marker, plot_ind=plot_ind)
        else:
            marker, hollow, color, size = marker
            x, y, z = subdata
            plt.plot([x[0], x[-1] + 2], [y[0], y[0]], color=color, label='1-core')
            plt.plot([x[0], x[-1] + 2], [y[2], y[2]], '--', color=color, label='4-cores')
        
    kwargs['plotfun'] = plot_intercept
    kwargs['crossover'] = True
    plot_data_scaling(*args, **kwargs)
    
def plot_data_simd(*args, **kwargs):
    kwargs['plot_simd'] = True
    allow_skip = kwargs.pop('allow_skip', False)
    ideal_efficiencies = {
        'srv2': 2.,
        'haswell': 4.
    }
    def const_func(state, x):
        return 1. if state['vectype'] == 'par' else 1. / ideal_efficiencies[state['descriptor']]
    def norm(data):
        for mech in mech_order:
            # select mechanisms that match this:
            subdata = [(i, x) for i, x in enumerate(data) if x[0]['mechdata.name'] == mech]
            # next, select the unvectorized SIMD as the baseline
            baseline = [d for d in subdata if d[1][0]['vectype'] == 'par']
            if allow_skip and len(baseline) == 0:
                continue
            assert len(baseline) == 1, 'Multiple unvectorized data'
            baseline = baseline[0]
            # assert baseline[1][1][1].size == 1, 'Multiple mechanism data'
            if baseline[1][1][1].size == 1:
                ymin = baseline[1][1][1][0]
                ymin_select=lambda y: ymin
            else:
                ymin_select=lambda y: baseline[1][1][1][:]
            for subi, subd in subdata:
                    data[subi] = scaling(subd, const_func=const_func, ymin_select=ymin_select)
        return data
                
    kwargs['norm'] = norm
    plot_data_standard(*args, **kwargs)

In [ ]:
class recursive_dict(object):
    def __init__(self, key, value_list, return_index=True):
        self.key = key
        self.value_list = None
        if value_list is not None:
            self.value_list = value_list.copy()
        self.return_index = return_index
        
    def __call__(self, state, **kwargs):
        key = self.key
        if 'key' in kwargs:
            key = kwargs.pop('key')
        values = self.value_list
        if 'values' in kwargs:
            values = kwargs.pop('values')
        if values is None:
            # no value list ->
            return key
        if isinstance(values, dict):
            # recursively defined
            k, v = values[state[key]]
            return self.__call__(state, key=k, values=v)
        if self.return_index:
            # if we want the index
            return values.index(state[key])
        # else the value
        return values[values.index(state[key])]

class marker_dict(object):
    def __init__(self, color_key, color_dict, marker_key, marker_dict,
                 size_key=None, size_dict=None, grey=False, num_colors=2,
                 rdict=recursive_dict, sizes=None):
        self.color_dict = rdict(color_key, color_dict, return_index=True)
        
        # handle color_maps
        cmap = 'Greys' if grey else 'inferno'
        self.internal_color_wheel = plt.get_cmap(cmap, num_colors + 1)
        self.color_wheel = lambda x: self.internal_color_wheel(x)
        self.num_colors = num_colors - 1
            
        self.marker_dict = rdict(marker_key, marker_dict)
        self.marker_wheel = wheeler(marker_wheel)
        
        if size_key is None:
            assert size_dict is None
            size_key = color_key
            size_dict = color_dict
        else:
            assert size_dict
            
            
        self.size_dict = rdict(size_key, size_dict)
        if sizes is None:
            sizes = num_colors
        sizearr = [12, 16, 20, 24]
        sizes = np.linspace(sizearr[-sizes], sizearr[-1], sizes, endpoint=True)
        self.size_wheel = wheeler(sizes)
    
    def __getlegend__(self):
        return False
        
    def __getcolor__(self, state):
        ind = self.color_dict(state)
        # assert ind <= self.num_colors
        return tuple(self.color_wheel(ind)[:3],) + (1,)
    
    def __getmarker__(self, state):
        return self.marker_wheel(self.marker_dict(state))
    
    def __getsize__(self, state):
        return self.size_wheel(self.size_dict(state))
    
    def reset(self):
        pass
    
    def __call__(self, state):
        marker, hollow = self.__getmarker__(state)
        return marker, hollow, self.__getcolor__(state), self.__getsize__(state)

In [ ]:
class compound_namer(object):
    def __normalize(self, kvdict):
        outdict = kvdict.copy()
        for key, values in kvdict.items():
            if values is not None:
                # now, for each new kv pair, ensure we have a further dictionary
                if isinstance(values, dict):
                    # recursively normalize
                    outdict[key] = self.__normalize(values)
                else:
                    # or a (key, None) tuple
                    outdict[key] = (values, None)
        return outdict

    def __init__(self, **kvlist):
        self.keys = []
        for key, values in name_order(list(self.__normalize(kvlist).items())):
            self.keys.append(recursive_dict(key, values, False))
                    
    
    def add_key(self, state, key):
        return pretty_name(state, key(state))
            
    def add_keys(self, state):
        name = ''
        for key in self.keys:
            if name:
                name += ' - '
            name += self.add_key(state, key)
        return name
    
    def __call__(self, state):
        return self.add_keys(state)

In [ ]:
class tracking_dict(recursive_dict):
    def __init__(self, *args, **kwargs):
        super(tracking_dict, self).__init__(*args, **kwargs)
        self.maps = {}
    
    def __call__(self, state, **kwargs):
        key = self.key
        if 'key' in kwargs:
            key = kwargs.get('key')
        values = self.value_list
        if 'values' in kwargs:
            values = kwargs.get('values')
        assert not values is None, "Can't use with compound namer"
        if not isinstance(values, dict):
            # we've reached the end of recursion
            if self.return_index:
                # if we want the index
                value = values.index(state[key])
            else:
                # else the value
                value = values[values.index(state[key])]
            name = pretty_name(state, key)
            assert state[key] not in self.maps or self.maps[state[key]] == (value, name)
            self.maps[state[key]] = (value, name)
        return super(tracking_dict, self).__call__(state, **kwargs)
            
        

# an override of the marker dict that compresses legends
class legend_handler(marker_dict):
    def __init__(self, *args, **kwargs):
        kwargs['rdict'] = tracking_dict
        self.explicit = kwargs.pop('explicit', False)
        self.replacements = kwargs.pop('replacements', {})
        self.ignore = kwargs.pop('ignore', [])
        
        super(legend_handler, self).__init__(*args, **kwargs)
        
    def reset(self):
        self.color_dict.maps = {}
        self.marker_dict.maps = {}
        self.size_dict.maps = {}

    def __getlegend__(self):
        handles = []
        for key, (color_ind, name) in sorted(self.color_dict.maps.items(), key=lambda x: x[1][0]):
            if name in self.ignore:
                continue
            handles.append(mpatches.Patch(color=self.color_wheel(color_ind),
                                          label=self.replacements.get(name, name)))
        for key, (marker_ind, name) in sorted(self.marker_dict.maps.items(), key=lambda x: x[1][0]):
            if len(self.marker_dict.maps) == 1 and not self.explicit:
                continue
            if name in self.ignore:
                continue
            markersize = 15
            if key in self.size_dict.maps:
                markersize, _ = self.size_dict.maps[key]
                markersize = self.size_wheel(markersize)
            marker, hollow = self.marker_wheel(marker_ind)
            marker_dict = {'marker': marker,
                           'color': 'k',
                           'markeredgecolor': 'k',
                           'markersize': markersize}
            if hollow:
                marker_dict['markerfacecolor'] = 'None'
            handles.append(mlines.Line2D([], [], linestyle='None',
                                         label=self.replacements.get(name, name), **marker_dict))
        return handles
    
class crossover_handler(legend_handler):
    def __getlegend__(self):
        # get default
        handles = super(crossover_handler, self).__getlegend__()
        # remove shallow
        handles = [x for x in handles if x._label != 'Shallow']
        # add 1 & 4 core count
        handles.append(mlines.Line2D([], [], linestyle='-', label=r'Shallow - \num{1}-core', color='k'))
        handles.append(mlines.Line2D([], [], linestyle='--', label=r'Shallow - \num{4}-cores', color='k'))
        return handles

In [ ]:
class default_filters(object):
    def __init__(self, **filters):
        self.filters = dict(platform=lambda x: True,
                       lang=lambda x: True,
                       vectype=lambda x: True,
                       cores='1',
                       conp='conp',
                       order=lambda x: True,
                       vecwidth=lambda x: True,
                       descriptor=lambda x: True,
                       rates='hybrid')
        self.filters.update(filters)
    
    def __add__(self, other):
        newfilt = self.filters.copy()
        newfilt.update(other)
        return default_filters(**newfilt)

def get_filters(lang='opencl', cpu=True, vecwidth='8', source=True, sparse=True):
    base = default_filters(rates='fixed')
    if lang == 'opencl':
        base += {'lang': 'opencl', 'vecwidth': vecwidth}
    elif lang == 'c':
        base += {'lang': 'c', 'vectype': 'openmp'}
    if cpu is not None:
        base += {'order': ('C' if cpu else 'F')}
        if not cpu and vecwidth == '8':
            # change default
            base += {'vecwidth': '128'}
    if source is not None:
        base += {'rtype': ('spec' if source else 'jac')}
        if not source:
            base += {'sparse': 'sparse' if sparse else 'full'}
    return base

def get_filtered(base=None, **filters):
    adata = data
    if 'additional_data' in filters:
        add = filters.pop('additional_data')
        for mech in data:
            assert not any(isinstance(x, list) for x in add[mech])
            assert not any(isinstance(x, list) for x in data[mech])
        adata = combine([data.copy(), add.copy()])
    for mech in adata:
        assert not any(isinstance(x, list) for x in adata[mech])
    base = (base or default_filters()) + filters
    return get_filtered_data(adata, **base.filters)

In [ ]:
%pdb

In [ ]:
# begin source term plots
#1 POCL equivalency over all type

filtered = combine([
    get_filtered(
        get_filters(),
        platform='portable',
        descriptor=lambda x: x in ['srv2', 'haswell'],
        vecwidth='4',
        vectype='par',
        strict=True),
    get_filtered(
        get_filters(),
        platform='portable',
        descriptor=lambda x: x in ['srv2', 'haswell'],
        vectype=lambda x: x in ['d', 'w']),
    get_filtered(
        get_filters(lang='c'),
        descriptor=lambda x: x in ['srv2', 'haswell'])])

namer = compound_namer(descriptor=None, vectype=None)
marker = legend_handler('descriptor', descriptor_order,
                        'vectype', vectype_order,
                        'vectype', vectype_order,
                        sizes=4)

plot_data_standard(filtered, namer, marker,
    ylog=True,
    limits={'y': [1e-3, 1e0]},
    legend_loc=ll_mid_right_long,
    figname='pocl_source_nonorm.pdf')

plot_data_standard(filtered, namer, marker,
    norm=lambda data: normalize_per_key(data, 'descriptor'),
    #limits={'y': [1e-3, 1e0]},
    legend_loc=ll_mid_right_long,
    figname='pocl_source.pdf')

In [ ]:
# next intel vs OpenMP

def __get_data(desc, vectype=['w', 'par'], include_openmp=True, openmp_only=False):
    filtered = get_filtered(
                       get_filters(),
                       platform='intel',
                       vectype=lambda x: x in vectype,
                       descriptor=lambda x: x in desc)
    openmp = get_filtered(
                       get_filters(lang='c'),
                       descriptor=lambda x: x in desc)
    if not include_openmp:
        return filtered
    elif openmp_only:
        return openmp
    
    return combine([filtered, openmp])

namer = compound_namer(descriptor=None,
                       vectype=None)
marker = legend_handler(
    'descriptor', descriptor_order,
    'vectype', vectype_order,
    'vectype', vectype_order,
    sizes=4
    )

#plot_data_standard(__get_data(['srv2']), namer, marker, {
#    'title': title + ' - SSE4.2'},
#                   norm=normalize_per_mech)

#plot_data_standard(__get_data(['haswell']), namer, marker, {
#    'title': title + ' - AVX2'},
#                   norm=normalize_per_mech)

plot_data_standard(__get_data(['haswell', 'srv2']), namer, marker,
                  figname='intel_source_nonorm.pdf',
                  limits={'y': [5e-4, 1e0]},
                  ylog=True,
                  legend_loc=ll_mid_right)

plot_data_standard(__get_data(['haswell', 'srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'descriptor'),
                  figname='intel_source.pdf',
                  legend_loc=ll_mid_right,
                  stats=True)

print('unvec')
plot_data_standard(__get_data(['haswell', 'srv2'], vectype=['par'], include_openmp=False), namer, marker,
                   norm=normalize_per_mech,
                  legend_loc=ll_mid_right,
                  stats=True)

print('omp')
plot_data_standard(__get_data(['haswell', 'srv2'], openmp_only=True), namer, marker,
                   norm=normalize_per_mech,
                  legend_loc=ll_mid_right,
                  stats=True)

In [ ]:
# conp vs conv
filtered = combine([get_filtered(
                        get_filters(),
                        platform='intel',
                        conp=lambda x: True,
                        descriptor='haswell'),
                    get_filtered(
                        get_filters(lang='c'),
                        conp=lambda x: True,
                        descriptor='haswell')])
        

namer = compound_namer(conp=None, vectype=None)
marker = legend_handler(
    'conp', conp_order,
    'vectype', vectype_order)


plot_data_standard(filtered, namer, marker,
                   ylog=True,
                   figname='source_conpvsconv.pdf')

In [ ]:
# C vs F on cpu
filtered = combine([get_filtered(
                        get_filters(),
                        platform='intel',
                        vectype='w',
                        descriptor='haswell',
                        order='C'),
                    get_filtered(
                        get_filters(),
                        platform='intel',
                        vectype='w',
                        descriptor='haswell',
                        order='F'),
                     get_filtered(
                        get_filters(lang='c'),
                        descriptor='haswell',
                        order='C'),
                    get_filtered(
                        get_filters(lang='c'),
                        descriptor='haswell',
                        order='F')])
        

namer = compound_namer(order=None, vectype=None)
marker = legend_handler(
    'order', data_order,
    'vectype', vectype_order)


plot_data_standard(filtered, namer, marker,
    norm=lambda data: normalize_per_key(data, ('lang', 'vectype')),
    legend_loc=ll_mid_right,
    limits={'y': (0.65, 3)},
    figname='source_cvsf.pdf',
    stats=True)

In [ ]:
# hybrid vs fixed
filtered = combine([
    get_filtered(get_filters(),
                        platform='intel',
                        descriptor='haswell',
                        vectype='par',
                        vecwidth ='4',
                        rates = lambda x: True,
                        order='C'),
    get_filtered(get_filters(),
                        platform='intel',
                        descriptor='haswell',
                        vectype='w',
                        vecwidth ='8',
                        rates = lambda x: True,
                        order='C')])
        

namer = compound_namer(order=None, vectype=None)
marker = legend_handler(
    'rates', ['fixed', 'hybrid'],
    'vectype', vectype_order,
    num_colors=3)

plot_data_standard(filtered, namer, marker,
    legend_loc=legend_top,
    norm=normalize_per_mech)

In [ ]:
# effect of vector width
def __get_data(desc, **kwds):
    return get_filtered(get_filters(**kwds),
                        platform='intel',
                        descriptor=desc,
                        vectype=lambda x: x in ['w', 'par'],
                        vecwidth = lambda x: x in ['4', '8', '16'],
                        order='C')
        

namer = compound_namer(order=None, vectype=None)
marker = legend_handler(
    'vectype', vectype_order,
    'vecwidth', vecwidth_order,
    num_colors=3)

plot_data_standard(__get_data('haswell'), namer, marker,
    norm=normalize_per_mech,
    legend_loc=ll_mid_righter_long,
    figname='source_vector_width.pdf',
    stats=True)

plot_data_standard(__get_data('srv2'), namer, marker,
    norm=normalize_per_mech,
    legend_loc=ll_mid_righter_long)

In [ ]:
# NVIDIA
def __get_data(desc=['srv2-gpu', 'gpu'], vw=['64', '128'], order='F'):
    return get_filtered(
        get_filters(cpu=False),
        platform='nvidia',
        descriptor=lambda x: x in desc,
        vecwidth=lambda x: x in vw,
        order=order,
        vectype='w')

namer = compound_namer(descriptor=None, vecwidth=None)
marker = legend_handler('descriptor', gpu_desc_order,
                       'vecwidth', gpu_vecwidth_order)


print('speedup - both')
plot_data_gpu_standard(__get_data(), namer, marker,
                       norm=normalize_per_mech,
                       legend_loc=legend_top,
                       figname='gpu_source_speedup.pdf',
                       stats=True)

namer = compound_namer(**{'mechdata.name': None})
marker = legend_handler('mechdata.name', mech_order,
                        'vecwidth', gpu_vecwidth_order,
                        'descriptor', gpu_desc_order,
                        num_colors=4)

#c2075
print('scaling - srv2')
plot_data_gpu_scaling(__get_data(desc=['srv2-gpu'], vw=['128']), namer, marker,
                      plot_condition_scaling=True,
                      ylog=True,
                      xlog=True,
                      legend_loc=legend_top)
 
# k40
print('scaling - hpc')
plot_data_gpu_scaling(__get_data(desc=['gpu'], vw=['128']), namer, marker,
                       plot_condition_scaling=True,
                       ylog=True,
                       xlog=True,
                       legend_loc=legend_top,
                       figname='gpu_source_scaling.pdf')
plot_data_gpu_scaling(__get_data(desc=['gpu'], vw=['128'], order='C'), namer, marker,
                       plot_condition_scaling=True,
                       ylog=True,
                       xlog=True,
                       legend_loc=legend_top)

In [ ]:
# C vs F on GPU
filtered = combine([get_filtered(
                        get_filters(cpu=False),
                        descriptor=lambda x: 'gpu' in x,
                        order='C'),
                    get_filtered(
                        get_filters(cpu=False),
                        descriptor=lambda x: 'gpu' in x,
                        order='F')])
        
namer = compound_namer(order=None, descriptor=None)
marker = legend_handler(
    'order', data_order,
    'descriptor', gpu_desc_order)


plot_data_gpu_standard(filtered, namer, marker,
    figname='source_gpu_cvsf.pdf',
    legend_loc=legend_top,
    stats=True)

plot_data_gpu_standard(filtered, namer, marker,
    norm=lambda data: normalize_per_key(data, 'descriptor'),
    legend_loc=legend_top,
    stats=True)

In [ ]:
%pdb

In [ ]:
# parallel scaling efficiency -- source terms
def __get_data(desc, vecwidth='8'):
    return combine([get_filtered(
                       get_filters(),
                       platform='intel',
                       vectype=lambda x: x in ['w'],
                       descriptor=lambda x: x in desc,
                       cores=lambda x: True,
                       vecwidth=vecwidth),
                    get_filtered(
                       get_filters(lang='c'),
                       descriptor=lambda x: x in desc,
                       cores=lambda x: True),
                   ])

namer = compound_namer(**{'mechdata.name': None, 'vectype':None})
marker = legend_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    num_colors=4)

print('haswell - source')
plot_data_scaling(__get_data(['haswell']), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (2e-1, 1.05),
                          'x': (None, 18)},
                  figname='source_parallel_scaling.pdf',
                  stats=True)

plot_data_scaling(__get_data(['haswell']), namer, marker,
                  legend_loc=legend_top,
                  scale=False,
                  limits={'x': (0, 18)},
                  ylog=True)

print('srv2 - source')
plot_data_scaling(__get_data(['srv2'], vecwidth='8'), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (4e-1, 1.05),
                          'x': (None, 10)})

marker = crossover_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    num_colors=4)
plot_data_scaling_crossover(__get_data(['haswell']), namer, marker,
                              legend_loc=legend_top,
                              ylog=True,
                              scale=False,
                              limits={'x': (None, 18)},
                           figname='source_crossover.pdf')

In [ ]:
# SIMD efficiency -- source terms
def __get_data(desc, vw='8'):
    return get_filtered(
                       get_filters(),
                       platform='intel',
                       vectype=lambda x: x in ['w', 'par'],
                       descriptor=lambda x: x in desc,
                       vecwidth=vw)

namer = compound_namer(**{'mechdata.name': None, 'vectype':None})
marker = legend_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    'vectype', vectype_order,
    num_colors=4,
    sizes=3)

print('haswell - source')
plot_data_simd(__get_data(['haswell']), namer, marker,
              legend_loc=legend_top,
              limits={'y': (5.5e-1, None)},
              figname='source_simd_efficiency.pdf',
              stats=True)

print('srv2 - source')
plot_data_simd(__get_data(['srv2']), namer, marker,
               legend_loc=legend_top,
               limits={'y': (None, 1.5)},
               figname='source_sse_simd_efficiency.pdf',
               stats=True)

In [ ]:
# sparse vs dense
def __get_data(desc):
    return combine([get_filtered(
                       get_filters(source=False),
                       platform='intel',
                       vectype='w',
                       descriptor=lambda x: x in desc,
                       sparse=lambda x: True),
                    get_filtered(
                       get_filters(source=False, lang='c'),
                       descriptor=lambda x: x in desc,
                       sparse=lambda x: True)
                   ])

namer = compound_namer(vectype=None,
                       sparse=None,
                       descriptor=None)
marker = legend_handler(
    'sparse', sparse_order,
    'vectype', vectype_order
    )

print('srv2 - langnorm')
plot_data_standard(__get_data(['srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'lang'),
                   legend_loc=ll_mid_right,
                   stats=True)

print('srv2 - nonorm')
plot_data_standard(__get_data(['srv2']), namer, marker,
                   legend_loc=ll_mid_right,
                   ylog=True,
                   figname='sparse_vs_dense_sse.pdf')

plot_data_standard(__get_data(['srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'sparse'),
                   legend_loc=ll_mid_right,
                   stats=True,
                   figname='sparse_vs_dense_see_speedup.pdf')

print('haswell - langnorm')
plot_data_standard(__get_data(['haswell']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'lang'),
                   legend_loc=ll_mid_right,
                   
                   stats=True)

print('haswell - nonorm')
plot_data_standard(__get_data(['haswell']), namer, marker,
                   legend_loc=ll_mid_right,
                   ylog=True,
                   figname='sparse_vs_dense.pdf')

plot_data_standard(__get_data(['haswell']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'sparse'),
                   legend_loc=ll_mid_right,
                   stats=True,
                   figname='sparse_vs_dense_speedup.pdf')

In [ ]:
# parallel scaling efficiency -- sparse jacobian
def __get_data(desc, sparse=True):
    return combine([get_filtered(
                       get_filters(source=False, sparse=sparse),
                       platform='intel',
                       vectype=lambda x: x in ['w'],
                       descriptor=lambda x: x in desc,
                       cores=lambda x: True),
                    get_filtered(
                       get_filters(source=False, lang='c', sparse=sparse),
                       descriptor=lambda x: x in desc,
                       cores=lambda x: True),
                   ])

namer = compound_namer(**{'mechdata.name': None, 'vectype':None})
marker = legend_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    num_colors=4)

# sparse
print('sparse haswell')
plot_data_scaling(__get_data(['haswell']), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (5e-1, 1.05),
                          'x': (None, 18)},
                  figname='sparse_jac_scaling.pdf',
                  stats=True)

print('sparse srv2')
plot_data_scaling(__get_data(['srv2']), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (5e-1, 1.05),
                          'x': (None, 9)},
                  stats=True)

# dense
print('dense haswell')
plot_data_scaling(__get_data(['haswell'], sparse=False), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (0.000000001, 1.05),
                          'x': (None, 18)},
                  figname='dense_jac_scaling.pdf',
                  stats=True)

print('dense srv2')
plot_data_scaling(__get_data(['srv2'], sparse=False), namer, marker,
                  legend_loc=legend_top,
                  limits={'y': (3.5e-1, 1.05),
                          'x': (None, 9)},
                  stats=True)

print('crossover sparse haswell')
marker = crossover_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    num_colors=4)
plot_data_scaling_crossover(__get_data(['haswell'], sparse=True), namer, marker,
                              legend_loc=legend_top,
                              ylog=True,
                              scale=False,
                              limits={'x': (None, 18)})

print('crossover dense haswell')
plot_data_scaling_crossover(__get_data(['haswell'], sparse=False), namer, marker,
                              legend_loc=legend_top,
                              ylog=True,
                              scale=False,
                              limits={'x': (None, 18)})

In [ ]:
# SIMD efficiency -- sparse jacobian
def __get_data(desc, sparse=True):
    return combine([get_filtered(
                       get_filters(source=False, sparse=sparse),
                       platform='intel',
                       vectype=lambda x: x in ['w', 'par'],
                       descriptor=lambda x: x in desc),
                   ])

namer = compound_namer(**{'mechdata.name': None, 'vectype':None})
marker = legend_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    num_colors=4)

# sparse
print('sparse haswell')
plot_data_simd(__get_data(['haswell']), namer, marker,
                  legend_loc=legend_top,
                  figname='sparse_jac_simd_efficiency.pdf')

print('sparse srv2')
plot_data_simd(__get_data(['srv2']), namer, marker,
                  legend_loc=legend_top)

# dense
print('dense haswell')
plot_data_simd(__get_data(['haswell'], sparse=False), namer, marker,
                  legend_loc=legend_top,
                  figname='dense_jac_scaling_simd_efficiency.pdf')

print('dense srv2')
plot_data_simd(__get_data(['srv2'], sparse=False), namer, marker,
                  legend_loc=legend_top)

In [ ]:
%pdb

In [ ]:
# gpu sparse vs dense
# sparse vs dense
def __get_data(desc, sparsetest=lambda x: True):
    return get_filtered(
                       get_filters(source=False, cpu=False),
                       platform='nvidia',
                       vectype='w',
                       descriptor=lambda x: x in desc,
                       sparse=sparsetest)

namer = compound_namer(sparse=None,
                       descriptor=None)
marker = legend_handler(
    'descriptor', gpu_desc_order,
    'sparse', sparse_order,
    )


plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu']), namer, marker,
    norm=lambda data: normalize_per_key(data, 'sparse'),
    figname='gpu_jacobian_speedup.pdf',
    legend_loc=legend_top,
    stats=True)

namer = compound_namer(**{'mechdata.name': None})
marker = legend_handler('mechdata.name', mech_order,
                        'sparse', sparse_order,
                        'mechdata.name', mech_order,
                        num_colors=4,
                        sizes=4)
# plot scaling for both:
print('hpc - both')
plot_data_gpu_scaling(__get_data(desc=['gpu']), namer, marker,
    plot_condition_scaling=True,
    ylog=True,
    xlog=True,
    legend_loc=legend_top,
    figname='gpu_sparse_vs_dense.pdf')

print('hpc - full')
plot_data_gpu_scaling(__get_data(desc=['gpu'], sparsetest=lambda x: x == 'full'), namer, marker, 
    plot_condition_scaling=True,
    ylog=True,
    xlog=True,
    legend_loc=legend_top)

print('srv2 - both')
plot_data_gpu_scaling(__get_data(desc=['srv2-gpu']), namer, marker, 
    plot_condition_scaling=True,
    ylog=True,
    xlog=True,
    legend_loc=legend_top)

print('srv2 - full')
plot_data_gpu_scaling(__get_data(desc=['srv2-gpu'], sparsetest=lambda x: x == 'full'), namer, marker, 
    plot_condition_scaling=True,
    ylog=True,
    xlog=True,
    legend_loc=legend_top)

In [ ]:
# next intel vs OpenMP
def __get_data(desc):
    return combine([get_filtered(
                       get_filters(source=False),
                       platform='intel',
                       vectype=lambda x: x in ['w', 'par'],
                       descriptor=lambda x: x in desc),
                    get_filtered(
                       get_filters(source=False, lang='c'),
                       descriptor=lambda x: x in desc)
                   ])

namer = compound_namer(descriptor=None,
                       vectype=None)
marker = legend_handler(
    'descriptor', descriptor_order,
    'vectype', vectype_order,
    sizes=2
    )


print('srv2 - sparse')
plot_data_standard(__get_data(['srv2']), namer, marker,
                   norm=normalize_per_mech)

print('haswell - sparse')
plot_data_standard(__get_data(['haswell']), namer, marker,
                   norm=normalize_per_mech)

print('both - sparse')
plot_data_standard(__get_data(['haswell', 'srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'descriptor'),
                   legend_loc=legend_top)

In [ ]:
# next intel vs OpenMP FD vs analytical
def __get_data(desc, sparse='sparse'):
    return combine([get_filtered(
                       get_filters(source=False),
                       platform='intel',
                       vectype='w',
                       rtype='jac',
                       sparse=sparse,
                       descriptor=lambda x: x in desc),
                    get_filtered(
                       get_filters(source=False),
                       platform='intel',
                       vectype='par',
                       vecwidth='4',
                       rtype='fdjac',
                       sparse=sparse,
                       descriptor=lambda x: x in desc),
                    get_filtered(
                       get_filters(source=False, lang='c'),
                       rtype=lambda x: 'jac' in x,
                       sparse=sparse,
                       descriptor=lambda x: x in desc)
                   ])

namer = compound_namer(descriptor=None,
                       vectype=None,
                       rtype=None)
marker = legend_handler(
    'rtype', jac_order,
    'vectype', vectype_order,
    sizes=2
    )


plot_data_standard(__get_data(['srv2']), namer, marker,
    norm=normalize_per_mech)

print('hpc - sparse')
plot_data_standard(__get_data(['haswell']), namer, marker,
    norm=lambda data: normalize_per_key(data, 'lang'),
    figname='finite_difference_vs_analytical.pdf',
    legend_loc=legend_top,
    ylog=True,
    limits = {'y': (0.5, 100),
              'x': (0, 900)},
    stats=True)

print('hpc - dense')
plot_data_standard(__get_data(['haswell'], sparse='full'), namer, marker,
    norm=lambda data: normalize_per_key(data, 'lang'),
    legend_loc=legend_top,
    ylog=True,
    limits = {'y': (0.5, 350)},
    stats=True)

In [ ]:
# gpu sparse vs dense
# sparse vs dense
def __get_data(desc, sparse='sparse'):
    return get_filtered(
                       get_filters(source=False, cpu=False),
                       platform='nvidia',
                       vectype='w',
                       sparse=sparse,
                       rtype=lambda x: 'jac' in x,
                       descriptor=lambda x: x in desc)

namer = compound_namer(rtype=None,
                       descriptor=None)
marker = legend_handler(
    'descriptor', gpu_desc_order,
    'rtype', jac_order,
    sizes=2
    )


plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu']), namer, marker,
                       norm=lambda data: normalize_per_key(data, 'descriptor'),
                       figname='finite_difference_vs_analytical_gpu.pdf',
                       stats=True,
                       legend_loc=legend_top)

plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu'], 'full'), namer, marker,
                       norm=lambda data: normalize_per_key(data, 'descriptor'),
                       stats=True)

In [ ]:
# C vs F
# next intel vs OpenMP FD vs analytical
def __get_data(desc):
    return combine([get_filtered(
                       get_filters(source=False),
                       platform='intel',
                       vectype='w',
                       order=lambda x: True,
                       descriptor=lambda x: x in desc),
                    get_filtered(
                       get_filters(source=False, lang='c'),
                       order=lambda x: True,
                       descriptor=lambda x: x in desc)
                   ])

namer = compound_namer(descriptor=None,
                       vectype=None,
                       order=None)
marker = legend_handler(
    'order', data_order,
    'vectype', vectype_order,
    sizes=2
    )


plot_data_standard(__get_data(['srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'lang'))

plot_data_standard(__get_data(['haswell']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'lang'))

plot_data_standard(__get_data(['haswell', 'srv2']), namer, marker,
                   norm=lambda data: normalize_per_key(data, 'descriptor'))

In [ ]:
# gpu C vs F
def __get_data(desc, sparse=True):
    return get_filtered(
                       get_filters(source=False, cpu=False, sparse=sparse),
                       platform='nvidia',
                       vectype='w',
                       order=lambda x: True,
                       descriptor=lambda x: x in desc)

namer = compound_namer(order=None,
                       descriptor=None)
marker = legend_handler(
    'order', data_order,
    'descriptor', gpu_desc_order,
    sizes=2
    )


plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu']), namer, marker,
    norm=lambda data: normalize_per_key(data, 'descriptor'),
    ylog=True)

plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu'], sparse=False), namer, marker,
    norm=lambda data: normalize_per_key(data, 'descriptor'),
    ylog=True)

In [ ]:
%pdb

In [ ]:
# gpu conp vs conv
def __get_data(desc):
    return get_filtered(
                       get_filters(source=False, cpu=False),
                       platform='nvidia',
                       vectype='w',
                       conp=lambda x: True,
                       descriptor=lambda x: x in desc)

namer = compound_namer(conp=None,
                       descriptor=None)
marker = legend_handler(
    'conp', conp_order,
    'descriptor', gpu_desc_order,
    sizes=2
    )


plot_data_gpu_standard(__get_data(['srv2-gpu', 'gpu']), namer, marker,
    norm=lambda data: normalize_per_key(data, 'descriptor'))

In [ ]:
# finally pyjac v2 vs v1
# it's to manually create the data objects for the old data
import os
from os.path import join, isdir, isfile
import yaml
v1_data = join(os.getcwd(), 'pyjac-v1')
v2_data = join(os.getcwd(), 'performance')
assert isdir(v1_data)
assert isdir(v2_data)
v1_runs = {}
for mech in [join(v1_data, x) for x in os.listdir(v1_data) if isdir(join(v1_data, x))]:
    # load data from v2
    mbase = os.path.basename(mech)
    with open(join(v2_data, mbase, mbase + '.yaml'), 'r') as f:
        mech_info = mechdata(**yaml.load(f))
    v1_runs[mbase] = []
    for file in [join(mech, x) for x in os.listdir(mech) if isfile(join(mech, x)) and x.endswith('.txt')]:
        if not '_nco_' in file:
            # mechanism
            continue
        if not '_nosmem_' in file:
            continue
        with open(file, 'r') as f:
            lines = f.readlines()
        mdata = []
        for line in lines:
            try:
                num_cond, runtime = [float(x) for x in line.split(',')]
            except:
                continue
            mdata.append(rundata(num_cond, -1, -1, runtime))
        if not len(mdata):
            continue
        rtype = 'jac' if 'fd' not in file else 'fdjac'
        sparse = 'full'
        lang = os.path.basename(file).split('_')[0]
        vecwidth = '128' if 'cuda' in file else 'False'
        order = 'F' if 'cuda' in file else 'C'
        vectype = 'w' if 'cuda' in file else 'openmp'
        descriptor = 'v1-gpu' if 'cuda' in file else 'v1'
        rates = 'hybrid'
        kernels = 'fixed'
        cores = '1'
        conp = 'conp'
        platform = 'False'
        v1_runs[mbase].append(run(
            rtype, sparse, lang, vecwidth, order,
            vectype, platform, rates, kernels, cores, conp,
            descriptor, mdata, mech_info))
        assert not any(isinstance(x, list) for x in v1_runs[mbase])
        

In [ ]:
filtered = combine([get_filtered(
                        get_filters(source=False),
                        platform='intel',
                        vectype='w',
                        descriptor='srv2',
                        sparse='full'),
                    get_filtered(
                        get_filters(source=False, lang='c'),
                        descriptor='srv2',
                        sparse='full'),
                    get_filtered(
                        get_filters(source=False, lang='c'),
                        additional_data=v1_runs,
                        descriptor='v1',
                        rates='hybrid',
                        sparse='full')
                    ])

namer = compound_namer(descriptor=None, vectype=None)
marker = legend_handler('descriptor', ['srv2', 'v1'],
                        'vectype', vectype_order,
                        sizes=2,
                        replacements={r'\texttt{sse4.2}': r'\texttt{sse4.2-v2}'})

plot_data_standard(filtered, namer, marker,
    limits={'y': (1e-3, 1e2)},
    ylog=True,
    legend_loc=ll_mid_right_long,
    figname='v1_vs_v2.pdf',
    stats=True)

plot_data_standard(filtered, namer, marker,
    norm=normalize_per_mech,
    legend_loc=legend_top,
    stats=True,
    ylog=True)

In [ ]:
filtered = combine([get_filtered(
                        get_filters(source=False, cpu=False,),
                        platform='nvidia',
                        vectype='w',
                        descriptor='srv2-gpu',
                        sparse='full'),
                    get_filtered(
                        get_filters(source=False, cpu=False),
                        lang='cuda',
                        additional_data=v1_runs,
                        descriptor='v1-gpu',
                        rates='hybrid',
                        sparse='full')
                    ])

namer = compound_namer(descriptor=None, vectype=None)
marker = legend_handler('descriptor', ['srv2-gpu', 'v1-gpu'],
                        'vectype', vectype_order,
                        num_colors=2,
                        sizes=2,
                        replacements={r'\texttt{C2075}': r'\texttt{C2075-v2}'})

plot_data_gpu_standard(filtered, namer, marker,
    ylog=True,
    legend_loc=ll_mid_right_long,
    figname='v1_vs_v2_gpu.pdf',
    stats=True)

plot_data_gpu_standard(filtered, namer, marker,
    legend_loc=ll_mid_right_long,
    norm=normalize_per_mech,
    stats=True)

In [ ]:
# curve fits
def __get_data(desc='haswell', sparse='sparse', source=False):
    if source:
        sparse = 'full'
    return combine([get_filtered(
                       get_filters(source=source),
                       platform='intel',
                       vectype='w',
                       sparse=sparse,
                       descriptor=lambda x: x in desc),
                    get_filtered(
                       get_filters(source=source, lang='c'),
                       sparse=sparse,
                       descriptor=lambda x: x in desc)
                   ])

def linear(x, a, b):
    return a * x ** b

from scipy.optimize import curve_fit
def plot_curvefit(*subdata, label=None, marker=None, plot_ind=None):
    gp.plot(*subdata, label=label, marker=marker, plot_ind=plot_ind)
    marker, hollow, color, size = marker
    x, y, z = subdata
    popt, pcov = curve_fit(linear, [0] + x, [0] + y)
    xrange = np.arange(x[0], x[-1] + 2)
    yrange = linear(xrange, *popt)
    cname = 'orange' if np.allclose([0.92964400000000003, 0.41147899999999998, 0.145367, 1.0], color) else 'purple'
    print(label, cname, popt, pcov)
    plt.plot(xrange, yrange, color=color)
    
namer = compound_namer(descriptor=None,
                       vectype=None)
marker = legend_handler(
    'sparse', sparse_order,
    'vectype', vectype_order,
    num_colors=2
    )

print('hpc')
plot_data_standard(__get_data(sparse=lambda x: True), namer, marker,
    ylog=True,
    legend_loc=legend_top,
    plotfun=plot_curvefit)

print('srv2')
plot_data_standard(__get_data(desc='srv2', sparse=lambda x: True), namer, marker,
    ylog=True,
    legend_loc=legend_top,
    plotfun=plot_curvefit)

plot_data_standard(__get_data(source=True), namer, marker,
    ylog=True,
    legend_loc=legend_top,
    plotfun=plot_curvefit)

plot_data_standard(__get_data(source=True, desc='srv2'), namer, marker,
    ylog=True,
    legend_loc=legend_top,
    plotfun=plot_curvefit)

In [ ]:
# plot simd efficiency scaling
# it's to manually create the data objects for the scaling data
import os
import yaml
from os.path import join, isdir, isfile
data_path = join(os.getcwd(), 'scaling')
assert isdir(data_path)
scaling_runs = {}
mbase = os.path.basename(data_path)
for num_reactions in [join(data_path, x) for x in os.listdir(data_path) if isdir(join(data_path, x))]:
    # load data from scaling
    try:
        nr = int(os.path.basename(os.path.normpath(num_reactions)))
    except ValueError:
        pass
    mech_info = {
        'mech': '',
        'n_reactions': nr,
        'n_species': 360,
        'n_reversible': -1,
        'n_cheb': 0,
        'n_plog': 0,
        'name': 'IC5H11OH'
    }
    with open(os.path.join(num_reactions, 'mech.yaml'), 'w') as file:
        yaml.dump(mech_info, file, default_flow_style=False)

# and load data
runs = data_parser.parse_data(rebuild=False, directory=data_path, use_descriptor='haswell')
# and move all the runs into a dummy mechanism
runs = dict(IC5H11OH=flatten(runs))

namer = compound_namer(**{'mechdata.name': None, 'vectype':None})
marker = legend_handler(
    'mechdata.name', mech_order,
    'vectype', vectype_order,
    'vectype', vectype_order,
    num_colors=0,
    sizes=3,
    ignore=['IC$_5$H$_{11}$OH'])

# and then calculate the scalings
plot_data_simd(runs, namer, marker, allow_skip=True,
               figname='simd_efficiency_scaling.pdf',
               stats=True)

In [ ]:
%pdb